In [2]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(0);
gpu_boole = torch.cuda.is_available()

In [3]:
class generator(nn.Module):
    # initializers
    def __init__(self, input_channel):
        super(discriminator, self).__init__()
        #TODO
        self.reflPad1 = nn.ReflectionPad1d((3,3))
        self.conv1 = nn.Conv1d(input_channel, 512, kernel_size=7, stride=1)
        self.act = nn.LeakyReLU(0.2)
        
        self.convTrans1 = nn.ConvTranspose1d(512,256, kernel_size=16, stride=8, padding=4)
        self.convTrans2 = nn.ConvTranspose1d(256,128, kernel_size=16, stride=8, padding=4)
        self.convTrans3 = nn.ConvTranspose1d(128,64, kernel_size=4, stride=2, padding=1)
        self.convTrans4 = nn.ConvTranspose1d(64,32,  kernel_size=4, stride=2, padding=1)
        self.convTrans5 = nn.ConvTranspose1d(32,1, kernel_size=7, stride=1)
        self.out = nn.Tanh()

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        #TODO
        reflPad1_out = self.reflPad1(input)
        conv1_out = self.conv1(reflPad1_out)
        act_out = self.act(conv1_out)
        
        convTrans1_out = self.convTrans1(act_out)
        act1_out = self.act(convTrans1_out)
        
        convTrans2_out = self.convTrans2(act1_out)
        act2_out = self.act(convTrans2_out)
        
        convTrans3_out = self.convTrans3(act2_out)
        act3_out = self.act(convTrans3_out)
        
        convTrans4_out = self.convTrans4(act3_out)
        act4_out = self.act(convTrans4_out)
        reflPad2_out = self.reflPad1(act4_out)
        
        
        convTrans5_out = self.convTrans5(reflPad2_out)
        result = self.out(convTrans5_out)
        return result 

In [4]:
class discriminator(nn.Module):
    # initializers
    def __init__(self):
        super(discriminator, self).__init__()
        #TODO
        self.RefPad = nn.ReflectionPad1d((3,3))
        self.conv1 = nn.Conv1d(1, 16, kernel_size=15, stride=1)
        self.act = nn.LeakyReLU(0.2)
        
        self.conv2 = nn.Conv1d(16, 64, kernel_size=41, stride=4, padding=20, groups=4)
        self.conv3 = nn.Conv1d(64, 256, kernel_size=41, stride=4, padding=20, groups=16)
        self.conv4 = nn.Conv1d(256, 1024, kernel_size=41, stride=4, padding=20, groups=16)
        self.conv5 = nn.Conv1d(1024, 1024, kernel_size=5, stride=1, padding=2)
        self.conv6 = nn.Conv1d(1024, 1, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool1d(kernel_size=4, stride=2, padding=1)
        self.sig = nn.Sigmoid()

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        #TODO
        reflPad_out = self.RefPad(input)
        conv1_out = self.conv1(reflPad_out)
        act1_out = self.act(conv1_out)
        
        conv2_out = self.conv1(act1_out)
        act2_out = self.act(conv2_out)
        
        conv3_out = self.conv1(act2_out)
        act3_out = self.act(conv3_out)
        
        conv4_out = self.conv1(act3_out)
        act4_out = self.act(conv4_out)
        
        conv5_out = self.conv1(act4_out)
        act5_out = self.act(conv5_out)
        
        conv6_out = self.conv1(act5_out)
        resultt = self.pool(conv6_out)
        
        return result 

In [ ]:
#Training code:

gpu_boole = torch.cuda.is_available()
cnn_boole = True #set True for CNN reshaping

#TODO tune the hyper parameter carefully to achieve a nash equilibrium
#The initial hyper parameters are not ideal, you need to tune them to make things work.
k=100 
epochs = 10
batch_size = 32 
lr_g = 0.2
lr_d = 0.2
train_interval = 10

G = generator(k)
D = discriminator()

if gpu_boole:
    G = G.cuda()
    D = D.cuda()
    
#data loader:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
#        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True)

#D,G optimizers:
G_optimizer = optim.Adam(G.parameters(), lr=lr_g)
D_optimizer = optim.Adam(D.parameters(), lr=lr_d)

#loss definition(s):
BCE_loss = nn.l1_loss()

#training loop:
D_losses = []
G_losses = []
print("Training start!")
for epoch in range(epochs):
    batch_number = 0
    for x_, _ in train_loader:
        batch_number += 1
        #reshaping depending on your architecture class:
        if not cnn_boole:
            x_ = x_.view(batch_size,-1) #this reshape is needed for MLP class
        if gpu_boole:
            x_ = x_.cuda()
        
        
        mini_batch = x_.size()[0]
        y_real_ = torch.ones(mini_batch)
        y_fake_ = torch.zeros(mini_batch)
        if gpu_boole:
            y_real_ = y_real_.cuda()
            y_fake_ = y_fake_.cuda()
        z_ = torch.randn((mini_batch, k))
        if cnn_boole:
            z_ = z_.view(-1, k, 1, 1) #needed for CNN        
        if gpu_boole:
            z_ = z_.cuda()
        # TODO train discriminator D
        # Step 1 get prediction of D on real data x_ and calculate D_real_loss for real data
        D_pred_real = D.forward(x_)
        D_pred_real = torch.reshape(D_pred_real, (-1,))
        D_real_loss = BCE_loss(D_pred_real, y_real_)
        
        # Step 2 get prediction of D on fake data generated by generator based on z_
        # and calculate D_fake_loss for fake data
        fake_images = G.forward(z_)
        D_pred_fake = D.forward(fake_images)
        D_pred_fake = torch.reshape(D_pred_fake, (-1,))
        D_fake_loss = BCE_loss(D_pred_fake, y_fake_)
        
        
        # Step 3 calculate the overall loss for D and update weight. (we've done this for you)
        D_train_loss = D_real_loss + D_fake_loss
        D.zero_grad()
        D_train_loss.backward()
        D_optimizer.step()
        D_losses.append(D_train_loss.data.item())
        
        # TODO train generator G
        # Step 0 think about the collapse problem we mentioned in lectures
        # and how we deal with that. The hyperparameter train_interval might help.
        if batch_number%train_interval!=0:
            
            # Step 1 calculate a new z_ and get prediction of fake data generated by 
            # generator based on z_
            z_ = torch.randn((mini_batch, k))
            if cnn_boole:
                z_ = z_.view(-1, k, 1, 1) #needed for CNN        
            if gpu_boole:
                z_ = z_.cuda()
            fake_images = G.forward(z_)
            G_pred_fake = D.forward(fake_images)
            G_pred_fake = torch.reshape(G_pred_fake, (-1,))
            G_train_loss = BCE_loss(G_pred_fake, y_real_)

            # Step 2 calculate the train loss for generator and update weight (we've done this for you)
            G.zero_grad()
            G_train_loss.backward()
            G_optimizer.step()

            G_losses.append(G_train_loss.data.item())
    
    print('[%d/%d] - loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), epochs, torch.mean(torch.FloatTensor(D_losses)),
                                                                  torch.mean(torch.FloatTensor(G_losses))))

#Plotting:

#Losses:
plt.subplot(1, 2, 1)
plt.plot(D_losses)
plt.title("D Loss")
plt.xlabel("Batch number")
plt.ylabel("Loss")
plt.subplot(1, 2, 2)
plt.plot(G_losses)
plt.title("G Loss")
plt.xlabel("Batch number")
plt.ylabel("Loss");